## Imports

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.express as px
from bs4 import BeautifulSoup

### Carrega o arquivo 'remuneracao_servidores.csv' e cria um DataFrame a partir dele.

In [2]:
file_path = '../ds/remuneracao_servidores.csv'
rem_org = pd.read_csv(file_path, sep=',', header=0, decimal='.')

try:       
    rem_org.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("no unnamed column.")
        
rem_org = rem_org.sort_values(by=['ORGAO', 'ANO', 'MES'])
 
orgaos = rem_org["ORGAO"].drop_duplicates()
anos = rem_org["ANO"].drop_duplicates().sort_values(ascending=False)
meses = rem_org["MES"].drop_duplicates()

# Retira espaço inicial
rem_org.rename(columns={' REMUNERACAO LEGAL DEVIDA(R$)' : 'REMUNERACAO LEGAL DEVIDA(R$)'}, inplace=True)

# Plotagens

## Órgãos que mais gastam
### Remuneração Total por Mês

In [27]:
ano = 2019
df = rem_org.loc[(rem_org.ANO == ano), ['REMUNERACAO LEGAL TOTAL(R$)', 'MES', 'ANO', 'ORGAO']].groupby(by=['ORGAO', 'ANO', 'MES'], as_index=False).sum()

# df.iloc[0:12]
# df
fig = px.line(df, title='Remuneração Legal Total de cada orgão ('+str(ano)+')', x='MES', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
# fig = px.scatter(df, title='Remuneração Legal Total de cada orgão ('+str(ano)+')', x='MES', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO', trendline='ols')
fig.show()

df_top = df[df['REMUNERACAO LEGAL TOTAL(R$)'] > 10000000]
df_top

fig = px.line(df_top, title='Órgãos com o total de remunerações maior que 10.000.000 (10 milhões)', x='MES', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
# fig = px.scatter(df_top, title='Órgãos com o total de remunerações maior que 10.000.000 (10 milhões)', x='MES', y="REMUNERACAO LEGAL TOTAL(R$)", color='ORGAO')
fig.show()

### Busca 

In [5]:
ano = 2018
orgao = 'AMAZONPREV'
df = rem_org.loc[(rem_org.ORGAO == orgao) & (rem_org.ANO == ano), 
                 ['REMUNERACAO LEGAL TOTAL(R$)', 'MES', 'ANO', 'ORGAO']].groupby(by=['ANO', 'MES'], as_index=False).sum()

fig = px.line(df, title='Remuneração Legal Total - '+orgao+' - '+str(ano), x='MES', y='REMUNERACAO LEGAL TOTAL(R$)')
fig.show()

## Servidores Duplicados no mesmo órgão

### Busca (Órgão, Ano, Mês)

In [29]:
orgao = 'UEA'
ano = 2019
mes = 7

df = rem_org.loc[(rem_org.ORGAO == orgao) & (rem_org.ANO == ano) & (rem_org.MES == mes)]
duplicados = df[df.duplicated(['NOME'], keep=False)]
n_serv = int((duplicados['NOME'].count())/2)

print('Servidores duplicados:', n_serv)
duplicados
# rem_org

Servidores duplicados: 2


,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,ORGAO
3629853,MARIA BETANIA LEAL DE OLIVEIRA,ESCOLA SUPERIOR DE TECNOLOGIA,P.AREAS EXAT.E TEC.4A.CL-NS.104.17,P AREAS EXATAS,ESTATUTARIO,6987.22,0.0,6987.22,1609.35,5377.87,2019,7,UEA
3629854,MARIA BETANIA LEAL DE OLIVEIRA,ESCOLA SUP.DE TECNOLOGIA-EST,PROF.DOUTOR ADJ.C 40HS-PD.111.20,-,ESTATUTARIO,24932.86,0.0,24932.86,7322.29,17610.57,2019,7,UEA
3629884,MARIA DO PERPETUO SOCORRO R LIMA,ESCOLA SUP.DE ARTES E TUR.ESAT,PROFESSOR C3 ED-ESP-III,-,ESTATUTARIO,6148.20,0.0,6148.20,1295.07,4853.13,2019,7,UEA
3629885,MARIA DO PERPETUO SOCORRO R LIMA,ESCOLA SUP.DE ARTES E TUR.ESAT,PROFESSOR,-,ESTATUTARIO,6309.20,0.0,6309.20,1368.83,4940.37,2019,7,UEA


### Quantidade de servidores duplicados por órgão

In [7]:
df = rem_org[['NOME', 'ANO', 'MES', 'ORGAO']]
df_bool = df.duplicated()
duplicados = df[df_bool]
duplicados_count = duplicados.groupby(['ORGAO'], as_index=False)['NOME'].count()
duplicados_count.rename(columns={'NOME': 'N_SERVIDORES'})

,ORGAO,N_SERVIDORES
0,ADAF,1
1,ALFREDO DA MATA,300
2,AMAZONPREV,156256
3,ARSAM,49
4,CASA CIVIL DO GOVERNO,104
5,CASA MILITAR,8
6,CETAM,326
7,CGE,42
8,CGL,29
9,DETRAN,8


### Quantidade de servidores duplicados por órgão e ano

In [8]:
df = rem_org[['NOME', 'ANO', 'MES', 'ORGAO']]
df_bool = df.duplicated()
duplicados = df[df_bool]
duplicados_count = duplicados.groupby(['ORGAO', 'ANO'], as_index=False)['NOME'].count()
duplicados_count.rename(columns={'NOME': 'N_SERVIDORES'})

,ORGAO,ANO,N_SERVIDORES
0,ADAF,2017,1
1,ALFREDO DA MATA,2017,63
2,ALFREDO DA MATA,2018,111
3,ALFREDO DA MATA,2019,117
4,ALFREDO DA MATA,2020,9
...,...,...,...
136,UEA,2019,28
137,UEA,2020,2
138,UGPE,2017,3
139,VICE-GOVERNADORIA,2017,1


### Quantidade de servidores duplicados por órgão, ano e mês

In [9]:
df = rem_org[['NOME', 'ANO', 'MES', 'ORGAO']]
df_bool = df.duplicated()
duplicados = df[df_bool]
duplicados_count = duplicados.groupby(['ORGAO', 'ANO', 'MES'], as_index=False)['NOME'].count()
duplicados_count.rename(columns={'NOME': 'N_SERVIDORES'})

,ORGAO,ANO,MES,N_SERVIDORES
0,ADAF,2017,8,1
1,ALFREDO DA MATA,2017,5,7
2,ALFREDO DA MATA,2017,6,7
3,ALFREDO DA MATA,2017,7,7
4,ALFREDO DA MATA,2017,8,7
...,...,...,...,...
791,UGPE,2017,11,1
792,UGPE,2017,12,1
793,UGPE,2017,13,1
794,VICE-GOVERNADORIA,2017,11,1


## Servidores Duplicados em órgãos diferentes

In [10]:

df = rem_org.loc[:,['NOME', 'ANO', 'MES', 'ORGAO']]

df = df.drop_duplicates(['NOME','ORGAO'])
df = df.groupby('NOME', as_index=False)[['ORGAO']].count()

df_bool = df[df['ORGAO'] > 1]
df_bool

nomes = df_bool['NOME']
nomes = rem_org.merge(nomes, indicator=True, how='inner')['NOME']
nomes.drop_duplicates()
# vai = rem_org[rem_org['NOME'] == nomes]

0                 AIRTON GONCALVES DOS SANTOS
36          ALEXANDRE HENRIQUE FREITAS ARAUJO
72                  AMILCAR DA SILVA FERREIRA
179         BARBARA BRANDAO FERREIRA SILVEIRA
216               DERLANI JOSE PEREIRA SERRAO
                         ...                 
541062          CIMARA BARROSO BRAGA DA SILVA
541080          DENIRA ISABEL GONCALVES SILVA
541107    MARA CRISTINA BATISTA DO NASCIMENTO
541135                   MARCEL PAREDES VALIN
541142                  THIRSO DEL CORSO NETO
Name: NOME, Length: 11042, dtype: object

### Busca de servidores

In [32]:
ano = 2018
serv_id = 36
registros_serv = rem_org.loc[(rem_org.NOME == nomes[serv_id]), :]
fig = px.line(registros_serv, title='Remuneração de '+nomes[serv_id]+', 2018', y='REMUNERACAO LEGAL TOTAL(R$)', x='ANO', color='ORGAO')
# fig = px.line(registros_serv, title='Remuneração de '+nomes[serv_id]+', 2018', y='FUNCAO', x='ANO', color='ORGAO')
# teste = rem_org.loc[(rem_org.NOME == nomes[12]) & (rem_org.ANO == ano), :]
# fig = px.line(teste, y='REMUNERACAO LEGAL TOTAL(R$)', x='MES', color='ORGAO')

fig.show()
registros_serv

,NOME,LOTACAO,CARGO,FUNCAO,VINCULO,REMUNERACAO LEGAL TOTAL(R$),DESC.TETO(R$),REMUNERACAO LEGAL DEVIDA(R$),DESCONTOS LEGAIS(R$),LIQUIDO DISPONIVEL(R$),ANO,MES,ORGAO
168751,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,13213.28,0.0,13213.28,1304.20,11909.08,2017,5,ADAF
166724,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,11458.28,0.0,11458.28,2769.54,8688.74,2017,10,ADAF
169725,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,19358.46,0.0,19358.46,5014.95,14343.51,2019,1,ADAF
170952,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,2,ADAF
170544,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,3,ADAF
168614,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,4,ADAF
169305,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,5,ADAF
170273,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,6,ADAF
169029,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,7,ADAF
166860,ALEXANDRE HENRIQUE FREITAS ARAUJO,GABINETE,ENGENHEIRO AGRONOMO,DIRETOR,ESTATUTARIO,14606.15,0.0,14606.15,3708.07,10898.08,2019,8,ADAF
